In [ ]:
from rdkit import Chem
import pandas as pd
from joblib import Parallel,delayed
from rdkit.Chem import AllChem
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')

In [ ]:
def runrxn(rxnsmarts,reactants):
    products=[]
    try:
        rxn=AllChem.ReactionFromSmarts(rxnsmarts)
        AllChem.SanitizeRxn(rxn)
    except:
        return False
    try:
        ps=rxn.RunReactants(reactants)
        for i in range(len(ps)):
            for j in range(len(ps[i])):
                products.append(Chem.MolToSmiles(ps[i][j]))
        return products
    except:
        return False

In [ ]:
def check_iso(products,all_prod):
    new_prod=[]
    overall_can=[]
    red_prod=[p for p in products if (p!=[] and p!= False)]
    for x in red_prod:
        can_prod=[]
        for prod in x:
            try:
                smiles=Chem.CanonSmiles(prod)
                if smiles in overall_can: #Check for repetition across x's Eg, [[a,b],[a,c]]
                    continue
                else:
                    overall_can.append(smiles)
                    if smiles in all_prod: #Checks if any element is  part of previous generations
                        can_prod.append(str(all_prod.index(smiles)))
                    else:
                        all_prod.append(smiles)
                        can_prod.append(smiles)
            except:
                continue
        if can_prod !=[]: # Checks for repetition of x's Eg, [[a,b],[c,d],[a,b]]
            min_prod=list(set(can_prod)) #Checks for repetition in x eg.[[a,a],[b,c]]
            new_prod.append(min_prod)
        else:
            continue

    return new_prod,all_prod

In [ ]:
reaction=pd.read_csv('FinalTemplate_MapRN.csv')
reactlist=reaction['Reaction Smarts'].tolist()

In [ ]:
#reactants is a list of list of Smiles of the initial set of reactants
reactants=(Chem.MolFromSmiles('C1CCCC=C1'),Chem.MolFromSmiles('C(O)=O'))
all_product=['C1CCCC=C1','C(O)=O']
prodlist = Parallel(n_jobs=-1, verbose=1)(delayed(runrxn)(r,reactants) for r in reactlist)
f_products,all_product=check_iso(prodlist,all_product)
dict={}
dict['Gen111'] = f_products
df = pd.DataFrame({key: pd.Series(value) for key, value in dict.items()})
df.to_csv('Cyclohexene Prod/Cyclohexene_NoRepeat_MapRN_Acid.csv')

In [ ]:
for i in range(2,5):
        products=[]
        ctr=1
        for rxns in f_products:
            ind=0
            for prod in rxns:
                subst=()
                ind=ind+1
                if prod.isdigit():
                    continue
                else:
                    prodlist = Parallel(n_jobs=-1, verbose=1)(delayed(runrxn)(r,(Chem.MolFromSmiles(prod),)) for r in reactlist)
                    print(prod)
                    new_prod,all_product=check_iso(prodlist,all_product)
                    products.extend(new_prod)
                    string='Gen'+str(i)+str(ctr)+str(ind)
                    print(string,len(new_prod))
                    dict[string] = new_prod
                    df = pd.DataFrame({key: pd.Series(value) for key, value in dict.items()})
                    df.to_csv('Cyclohexene Prod/Cyclohexene_NoRepeat_MapRN_Acid.csv')

            ctr=ctr+1
        f_products=products
pd.DataFrame({'Cyclohexene  Products': all_product}).to_csv('Cyclohexene Prod/Cyclohexene_MapRN_Acid_AllProducts.csv')

In [ ]:
pd.DataFrame({'Cyclohexene  Products': all_product}).to_csv('Cyclohexene Prod/Cyclohexene_MapRN_Acid_AllProducts.csv')